In [1]:
using AutoGrad
using Deeplearning
using Plots
import CSV

┌ Info: Precompiling Deeplearning [0d5bc7a0-39dd-11e9-2aab-59c8d4b467de]
└ @ Base loading.jl:1186
ERROR: LoadError: UndefVarError: k not defined
Stacktrace:
 [1] top-level scope at none:0
 [2] include at ./boot.jl:326 [inlined]
 [3] include_relative(::Module, ::String) at ./loading.jl:1038
 [4] include(::Module, ::String) at ./sysimg.jl:29
 [5] top-level scope at none:2
 [6] eval at ./boot.jl:328 [inlined]
 [7] eval(::Expr) at ./client.jl:404
 [8] top-level scope at ./none:3
in expression starting at /Users/mertceylan/git/Deeplearning.jl/src/Deeplearning.jl:1


ErrorException: Failed to precompile Deeplearning [0d5bc7a0-39dd-11e9-2aab-59c8d4b467de] to /Users/mertceylan/.julia/compiled/v1.1/Deeplearning/gtjFf.ji.

In [ ]:
train = CSV.read("datasets/digit-recognizer/train.csv")
# test = CSV.read("datasets/digit-recognizer/test.csv")

In [ ]:
d,dr = @onehot train.label

In [ ]:
train_normalized = nothing
for i in 1:100
    normalized = reshape(train[i, 2:end]./(maximum(train[i,2:end])+0.1), (28,28,1,1))
    if train_normalized == nothing
        train_normalized = normalized
    else
        train_normalized = cat(train_normalized, normalized, dims=4)
    end
end

In [ ]:
train_gpu = train_normalized

In [ ]:
input = @cudaarray rand(28, 28, 1, 10)
w1 = ((rand(3,3,1,6).-0.5)./100)
w2 = ((rand(3,3,6,16).-0.5)./100)

In [ ]:
conv1 = convolve(input, w1, (1,1), (1,1))
x = conv1(input)
@show size(x)
pool1 = avgpool(x, (2,2), (2,2), (1,1))
x = pool1(x)
@show size(x)
# conv2 = convolve(x, w2, (1,1), (1,1))
# x = conv2(x)
# pool2 = avgpool(x, (2,2), (2,2), (1,1))
# x = pool2(x)
# dense1 = dense(x, 120)
# x = dense1(x)
# dense2 = dense(x, 84)
# x = dense2(x)
# dense3 = dense(x, 10)
# layers=[dense3,dense2,dense1,conv2,conv1]

In [ ]:
f(x) = softmax(dense3(relu(dense2(relu(dense1(pool2(relu(conv2(pool1(relu(conv1(x))))))))))))

In [ ]:
f(train_gpu)

In [ ]:
using BenchmarkTools

In [ ]:
@btime f(train_gpu)

In [ ]:
acc_plot = []
loss_plot = []

In [ ]:
dff(x,y) = @diff squared_diff(f(x), y)

for e = 1:1
    println("Epoch ", e)
    misfits=0
    mean_err=0
    for i=1:size(train_gpu)[4]
        o = f(train_gpu[:,:,:,i])
        olabel = @onehotdecode dr o
        
        if(olabel == nothing || olabel!=train[i,:label])
           misfits+=1 
        end
        
        err = squared_diff(o, d[train[i,:label]])
        mean_err+=err
        
        dv = dff(train_gpu[:,:,:,i], d[train[i,:label]])
        dparams = @parameters dv

        for pidx in 1:length(dparams[1:end-1])
            dw = grad(dv, dparams[pidx])
            layers[pidx].matrix .-=0.1*dw
        end
    end
    append!(acc_plot, (1-(misfits/size(train_gpu)[4]))*100)
    append!(loss_plot, mean_err/size(train_gpu)[4])
end

In [ ]:
println(acc_plot[end])
println(loss_plot[end])

In [ ]:
plot([acc_plot], linewidth=2, title="Accuracy")

In [ ]:
plot([loss_plot], linewidth=2, title="Loss")